In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from os import listdir
from os.path import isfile, join
import nltk

import wordCountSentimentFull as wordCountSentiment

In [2]:
dictionary = pd.read_csv('LoughranMcDonald_MyDictionary2.csv')
dictionary.drop('Unnamed: 0', 1, inplace=True)
dictionary = dictionary.fillna('')
wordCountSentiment.lemmatizeDictionary(dictionary)
allWords = list()
word2category = {}
for col in ['CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING']:
    allWords = allWords + dictionary[col].tolist()
    for w in dictionary[col]:
        word2category[w] = col
allWords.remove('')
allWords = list(set(allWords))


porter_stemmer = PorterStemmer()

#     tokens = word_tokenize(sentence)
#     sentence = [porter_stemmer.stem(x) for x in tokens]

In [3]:
tickers=['AAPL','AMZN','GOOGL','MSFT','FB','INTC','CSCO','CMCSA','NVDA','NFLX']
#tickers=['AAPL']

In [4]:
# EXTRACT SENTIMENT AGGREGATED BY TIMESPAN

mypath = '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'
tickFiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for ticker in tickers: 
    print(ticker)
    news =  pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/'+ticker+'.csv')
    count =1
    while str(ticker)+str(count)+'.csv' in tickFiles:
        print(ticker+str(count))
        newsTemp = pd.read_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/preprocessing/preprocessed/' + ticker +str(count)+'.csv')
        news = pd.concat([news, newsTemp])
        count+=1
    news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
    news = news.sort_values(by=['PUBLICATION_DATE'])
    news = news.reset_index(drop=True)
    news = news.drop(['Unnamed: 0'], axis=1)
    notNaN = news['SUMMARY'].dropna(how='any').index.tolist()
    NaN = []
    for i in news.index:
        if(i not in notNaN):
            NaN.append(i)
    news = news.drop(NaN)
    news = news.reset_index(drop=True)
    news['PUBLICATION_DATE'] = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S +%f') for row in news['PUBLICATION_DATE']]
    
    
    timeSpan = pd.read_csv("/home/simone/Desktop/NLFF/indexes/indexes"+ticker+".csv")
    timeSpan = timeSpan['date'].tolist()
    # This dataset is already GMT+0
    timeSpan = [datetime.strptime(row, '%Y-%m-%d %H:%M:%S') for row in timeSpan]
    sentiment = pd.DataFrame(columns=['initTime','num_news']+allWords)
    initDate = max(timeSpan[0], news['PUBLICATION_DATE'][0])
    finalDate = min(timeSpan[len(timeSpan)-1], news['PUBLICATION_DATE'][len(news)-1])
    i = 0;
    j = 0
    # ALLINEAMENTO INIZIO
    while(timeSpan[j] < initDate):
        j+=1
    init = j
    while(news['PUBLICATION_DATE'][i] <= initDate):
        i+=1
    print("Start: "+str(initDate) +" i: "+str(i)+" j: "+str(j))
    print("End: "+str(finalDate))
    #ALLINEAMENTO FINE DENTRO I WHILE
    while( news['PUBLICATION_DATE'][i] < finalDate and timeSpan[j] < finalDate ):
        num_sentiment = 0
        initTime = timeSpan[j]
        normal_sum = {w:0 for w in allWords+['initTime','num_news']}
        num_sentiment = {w:0 for w in allWords}
        num_news = 0
        while(i<len(news)-1 and timeSpan[j] > news['PUBLICATION_DATE'][i]):
            if not (timeSpan[j] > news['PUBLICATION_DATE'][i] and timeSpan[j-1] <= news['PUBLICATION_DATE'][i]):
                print("timeSpan["+str(j)+"]: "+str(timeSpan[j])+" news["+str(i)+"] : " +str(news['PUBLICATION_DATE'][i]) + " timeSpan["+str(j-1)+"]: "+str(timeSpan[j-1]))
                assert False
            try:
                tokens = word_tokenize(news['SUMMARY'][i])
                tokens = [porter_stemmer.stem(x) for x in tokens]
                for token in tokens:
                    if token in allWords:
                        normal_sum[token] +=1
                        num_sentiment[token] +=1
                num_news += 1
            except:
                print("ERROR news "+str(i))
            i+=1
            if(i%500 == 0):
                print(i)
        j+=1
        normal_sum = {f:normal_sum[f]/num_sentiment[f] if num_sentiment[f]>0 else 0 for f in allWords}
        normal_sum['initTime'] = initTime
        normal_sum['num_news'] = num_news
        sentiment.loc[j] = normal_sum
    
    #Normalize for max num words
    M = sentiment.max()[2:]
    for k in M.keys():
        M[k] += 1 if( M[k]==0) else 0
        sentiment[k] = sentiment[k]/M[k]
    
    #Multipy for factor of frequency of word
    num_intervals = {w:0 for w in allWords}
    for index, row in sentiment.iterrows():
        for k in num_intervals.keys():
            num_intervals[k] += 1 if sentiment[k].at[index]>0 else 0
    for k in num_intervals.keys():
        num_intervals[k] += 1 if num_intervals[k]==0 else 0
        sentiment[k] = sentiment[k]*np.log(len(sentiment)/num_intervals[k])
    #Compute mean "TF-IDF" frequency for all the words in a category
    sentiment2 = pd.DataFrame(columns=['initTime','CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING', 'NUM_NEWS'])
    for i, row in sentiment.iterrows():
        normal_sum = {'CONSTRAINING': 0, 'LITIGIOUS': 0, 'NEGATIVE': 0, 'POSITIVE': 0, 'UNCERTAINTY': 0,'SUPERFLUOUS':0,'INTERESTING':0}
        for k in allWords:
            normal_sum[word2category[k]] += row[k]
        for col in ['CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING']:
            normal_sum[col] = normal_sum[col]/ row['num_news'] if row['num_news']>0 else normal_sum[col]
        sentiment2.loc[i] = {'initTime':row['initTime'], 'CONSTRAINING':normal_sum['CONSTRAINING'],
                            'LITIGIOUS': normal_sum['LITIGIOUS'],
                            'NEGATIVE': normal_sum['NEGATIVE'], 'POSITIVE': normal_sum['POSITIVE'],
                            'UNCERTAINTY': normal_sum['UNCERTAINTY'], 'SUPERFLUOUS': normal_sum['SUPERFLUOUS'],
                            'INTERESTING': normal_sum['INTERESTING'], 'NUM_NEWS':row['num_news'] }
            
    sentiment2.to_csv('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/SentimentTF-IDF/'+ticker+'.csv')


AAPL
AAPL1
AAPL2
Start: 2017-04-03 17:27:34 i: 1 j: 3817
End: 2018-06-21 23:20:00
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
AMZN
AMZN1
AMZN2
Start: 2017-04-26 13:00:00 i: 1 j: 3925
End: 2018-06-22 02:35:18
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
GOOGL
GOOGL1
Start: 2017-05-21 15:00:05 i: 1 j: 4051
End: 2018-06-22 00:14:21
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13

In [ ]:
M = sentiment.max()
for k in M.keys():
    M[k] += 1 if( M[k]==0) else 0
    sentiment[k] = sentiment[k]/M[k]
    

In [ ]:
sentiment = pd.DataFrame(columns=['initTime','CONSTRAINING','LITIGIOUS','NEGATIVE','POSITIVE','UNCERTAINTY','SUPERFLUOUS','INTERESTING', 'NUM_NEWS'])


In [ ]:
dictionary